In [32]:
# CODE CELL 1: Setup, Imports, and Modular LineCounter Class

import cv2
import numpy as np
from ultralytics import YOLO

# --- CONFIGURATION ---
VIDEO_PATH = 'demofootfall.mp4'
MODEL_NAME = 'yolov8n.pt'
LINE_REF_PERCENT = 50
COUNT_DIRECTION = 'left_to_right'

class LineCounter:
    def __init__(self, line_coords: tuple, entry_direction: str):
        self.line_coords = line_coords
        self.entry_direction = entry_direction
        self.is_vertical = (line_coords[0][0] == line_coords[1][0])
        self.reference_coord = line_coords[0][0] if self.is_vertical else line_coords[0][1]
        self.counted_objects = {}
        self.entry_count = 0
        self.exit_count = 0

    def check_crossing(self, track_id: int, current_centroid: tuple) -> str or None:
        current_c = current_centroid[0] if self.is_vertical else current_centroid[1]

        if track_id not in self.counted_objects:
            self.counted_objects[track_id] = current_c
            return None

        prev_c = self.counted_objects[track_id]
        line = self.reference_coord
        
        straddled_line = (prev_c < line and current_c >= line) or \
                             (prev_c > line and current_c <= line)
        
        if straddled_line:
            movement = current_c - prev_c 
            is_entry = False
            
            if self.is_vertical:
                if (movement > 0 and self.entry_direction == 'left_to_right') or \
                   (movement < 0 and self.entry_direction == 'right_to_left'):
                    is_entry = True
            else: 
                if (movement > 0 and self.entry_direction == 'top_to_bottom') or \
                   (movement < 0 and self.entry_direction == 'bottom_to_top'):
                    is_entry = True

            if is_entry:
                self.entry_count += 1
                direction = "ENTRY"
            else:
                self.exit_count += 1
                direction = "EXIT"

            self.counted_objects[track_id] = current_c 
            return direction

        self.counted_objects[track_id] = current_c
        return None

    def get_counts(self) -> tuple:
        return self.entry_count, self.exit_count

In [33]:
# CODE CELL 2: Visualization Helper

def draw_info(frame, entries, exits, line_coords, frame_info):
    
    cv2.line(frame, line_coords[0], line_coords[1], (0, 0, 255), 3)

    cv2.putText(frame, f"ENTRIES: {entries}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f"EXITS: {exits}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    cv2.putText(frame, frame_info, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

In [34]:
model = YOLO(MODEL_NAME)
print(f"Loaded model: {MODEL_NAME}")

cap = cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened():
    print(f"Error: Could not open video file at {VIDEO_PATH}. Check file name.")
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    LINE_X_COORD = int(frame_width * (LINE_REF_PERCENT / 100))
    LINE_COORDS = ((LINE_X_COORD, 0), (LINE_X_COORD, frame_height))
    
    counter = LineCounter(LINE_COORDS, entry_direction=COUNT_DIRECTION) 
    
    frame_count = 0
    final_entries = 0
    final_exits = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1

        results = model.track(frame, 
                              persist=True, 
                              classes=[0],
                              tracker='bytetrack.yaml', 
                              verbose=False)
        
        if results and results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int) 
            track_ids = results[0].boxes.id.cpu().numpy().astype(int) 

            for box, track_id in zip(boxes, track_ids):
                x1, y1, x2, y2 = box
                
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                centroid = (center_x, center_y)

                crossing_status = counter.check_crossing(track_id, centroid)
                final_entries, final_exits = counter.get_counts()

                color = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.circle(frame, centroid, 5, color, -1)
                
                label = f"ID:{track_id}"
                if crossing_status:
                    label += f" | {crossing_status}"
                
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
                
            frame_info = f"Frame: {frame_count} | Model: {MODEL_NAME}"
            draw_info(frame, final_entries, final_exits, LINE_COORDS, frame_info)

            cv2.imshow('Footfall Counter (Press Q to Exit)', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

print("\n--- Processing Complete ---")
print(f"Total Entries: {final_entries}")
print(f"Total Exits: {final_exits}")

Loaded model: yolov8n.pt

--- Processing Complete ---
Total Entries: 1
Total Exits: 0
